In [1]:
import sys
import os
import pyqtgraph as pg
import numpy as np
from pyqtgraph.Qt import QtCore, QtGui
import pandas as pd
import bruges
import lasio
from welly import Well
import lxml


C:\Users\hoogec\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [50]:
curves_info = lxml.etree.parse(r'C:\Users\hoogec\Documents\GitHub\deerfoot\CurveData.xml').getroot()

curve_displ = [{'name': 'Son-Den', 'curves': [['GammaRay', 'SelfPot'], ['Sonic'], ['Density']]}]
curve_displ.append({'name': 'Son-Shear', 'curves': [['GammaRay', 'SelfPot'], ['Sonic'], ['ShearSonic']]})
curve_displ.append({'name': 'Resistivity', 'curves': [['GammaRay', 'SelfPot'], ['DeepRes', 'MedRes', 'ShallowRes']]})
curve_displ.append({'name': 'Porosity-Lime', 'curves': [['GammaRay', 'SelfPot'], ['DenPorLime', 'NeutPorLime']]})
curve_displ.append({'name': 'Porosity-Sand', 'curves': [['GammaRay', 'SelfPot'], ['DenPorSand', 'NeutPorSand']]})
curve_displ.append({'name': 'Porosity-Dol', 'curves': [['GammaRay', 'SelfPot'], ['DenPorDol', 'NeutPorDol']]})
curve_displ.append({'name': 'LMR', 'curves': [['GammaRay', 'SelfPot'], ['Lambda'], ['Mu'],['Density']]})
curve_displ.append({'name': 'MiniPlot-Lime', 'curves': [['GammaRay', 'SelfPot'], ['DeepRes', 'MedRes', 'ShallowRes'], ['DenPorLime', 'NeutPorLime']]})
curve_displ.append({'name': 'MiniPlot-Sand', 'curves': [['GammaRay', 'SelfPot'], ['DeepRes', 'MedRes', 'ShallowRes'], ['DenPorSand', 'NeutPorSand']]})
curve_displ.append({'name': 'MiniPlot-Dol', 'curves': [['GammaRay', 'SelfPot'], ['DeepRes', 'MedRes', 'ShallowRes'], ['DenPorDol', 'NeutPorDol']]})


In [99]:
#added by C. Hooge    
class helperFunctions:
    def CheckMnemonic(crv,lascrvs,curves_info):
        orig_mn = crv.mnemonic
        pos = [0]
        if crv.mnemonic != "DEPT":
            name,mnemonic = helperFunctions.Find_Curve_Data(crv.mnemonic,curves_info)
#        print('mnemonic check',orig_mn,name,mnemonic)
            if name is not None:
                txt_alias2 = ".//*[@name='" + name + "']/Aliases/Name"
                crv_fnd_all = curves_info.findall(txt_alias2)
                pos = [i for i,x in enumerate(crv_fnd_all) if x.text==orig_mn]
        
#            print("hereA",name,orig_mn,pos,len(pos),(name in lascrvs))
                       
                if mnemonic is not None:
                    if name in lascrvs:
                        if len(pos) > 0:
                            crv.mnemonic = name + "_" + str(pos[0])
                        else:
                            crv.mnemonic = name + "_0"
                    
#                crv.mnemonic = name
                    else:
                        crv.mnemonic = name
                
#        crv.original_mnemonic = orig_mn
        else:
            name = 'Depth'
            crv.mnemonic = name  
 
        crv.original_mnemonic = orig_mn
        
        return crv

    def Find_Curve_Data(crv_mn,curves_info,search='mnemonic'):
        if search == 'mnemonic':
            txt = 'Curve[Mnemonic="' + crv_mn + '"]'
            txt_alias = "Curve/Aliases[Name='" + crv_mn + "']"
        if (search == 'name'):
#            txt ='Curve[Name="' + crv + '"]'
            txt = "Curve[@name='" + crv_mn + "']"
            txt_alias = ""

        crv_fnd = curves_info.find(txt)
#        print('search',search,crv_mn,txt,crv_fnd)

        name = None
        mnemonic = None
        min1 = None
        max1 = None
        reversed1 = None
        units = None
        colour = None
        if crv_fnd is not None:
            name = crv_fnd.attrib['name']
            mnemonic = crv_fnd.findtext('Mnemonic')
            min1 = crv_fnd.findtext('Minimum')
            max1 = crv_fnd.findtext('Maximum')
            reversed1 = crv_fnd.findtext('Reversed')
            units = crv_fnd.findtext('Units')
            colour = crv_fnd.findtext('Colour')
        else:
            crv_fnd = curves_info.find(txt_alias)        
            if crv_fnd is not None:
                name = crv_fnd.getparent().attrib['name']
                mnemonic = crv_fnd.getparent().findtext('Mnemonic')
                min1 = crv_fnd.getparent().findtext('Minimum')
                max1 = crv_fnd.getparent().findtext('Maximum')
                reversed1 = crv_fnd.getparent().findtext('Reversed')
                units = crv_fnd.getparent().findtext('Units')
                colour = crv_fnd.getparent().findtext('Colour')
            
        return name, mnemonic, min1, max1, reversed1, units, colour
    
    def ConvertCurveToMetric(crv_data,conversion=1):
        return crv_data * conversion
        
    def str2bool(v):
        
        return v.lower() in ("yes", "true", "True", "t", "1")       

    def CheckCurvesDisplay(crvs_disp,curvesdict):
#        for cd in curve_displ:
#            print("crvs_disp",crvs_disp['curves'])
        new_crvs_list = []
        for cv in crvs_disp['curves']:        
            tst = set(cv).intersection(curvesdict)
#            print("cv",cv,tst,len(tst))

            if len(tst) > 0:
                new_crvs_list.append(list(tst))
#                    print(list(tst))
                        
            
#            print("cd NEW",new_crvs_list)
        return new_crvs_list


In [100]:
las = lasio.read('100051403911W400.las')
for crv in las.curves:
            if crv.mnemonic != "DEPT":
                name, mnemonic, min1, max1, reversed1, units,colour = helperFunctions.Find_Curve_Data(crv.mnemonic,curves_info,search='mnemonic')
                las.curves.minimum = min1
                las.curves.maximum = max1
                las.curves.reversed = reversed1
                las.curves.units = units
                las.curves.colour = colour
            
                if name is None:
                    name = crv.mnemonic

                crv.mnemonic = name


Found nonstandard LAS section: ~Tops


In [101]:
list(las.curvesdict)


['DEPT',
 'CaliperX',
 'CaliperY',
 'DenCorr',
 'DenPorDol',
 'DenPorLime',
 'DenPorSand',
 'DT_',
 'Sonic',
 'FocusedRes',
 'GammaRay',
 'MBTC',
 'MicroInverse',
 'MicroNormal',
 'NeutPorDol',
 'NeutPorLime',
 'NeutPorSand',
 'PhotoEffect',
 'DeepRes',
 'MedRes',
 'SelfPot']

In [102]:
curve_displ_new = []
for cd in curve_displ:
#    print(cd['name'])
    helperFunctions.CheckCurvesDisplay(cd,las.curvesdict)
    new_list = helperFunctions.CheckCurvesDisplay(cd, las.curvesdict)
    z = {**{'name': cd['name']}, **{'curves': new_list}}
    print('z',z)

    curve_displ_new.append(dict(z))


z {'name': 'Son-Den', 'curves': [['SelfPot', 'GammaRay'], ['Sonic']]}
z {'name': 'Son-Shear', 'curves': [['SelfPot', 'GammaRay'], ['Sonic']]}
z {'name': 'Resistivity', 'curves': [['SelfPot', 'GammaRay'], ['MedRes', 'DeepRes']]}
z {'name': 'Porosity-Lime', 'curves': [['SelfPot', 'GammaRay'], ['DenPorLime', 'NeutPorLime']]}
z {'name': 'Porosity-Sand', 'curves': [['SelfPot', 'GammaRay'], ['DenPorSand', 'NeutPorSand']]}
z {'name': 'Porosity-Dol', 'curves': [['SelfPot', 'GammaRay'], ['NeutPorDol', 'DenPorDol']]}
z {'name': 'LMR', 'curves': [['SelfPot', 'GammaRay']]}
z {'name': 'MiniPlot-Lime', 'curves': [['SelfPot', 'GammaRay'], ['MedRes', 'DeepRes'], ['DenPorLime', 'NeutPorLime']]}
z {'name': 'MiniPlot-Sand', 'curves': [['SelfPot', 'GammaRay'], ['MedRes', 'DeepRes'], ['DenPorSand', 'NeutPorSand']]}
z {'name': 'MiniPlot-Dol', 'curves': [['SelfPot', 'GammaRay'], ['MedRes', 'DeepRes'], ['NeutPorDol', 'DenPorDol']]}


In [95]:
curve_displ

[{'name': 'Son-Den', 'curves': [['SelfPot', 'GammaRay'], ['Sonic']]},
 {'name': 'Son-Shear', 'curves': [['SelfPot', 'GammaRay'], ['Sonic']]},
 {'name': 'Resistivity',
  'curves': [['SelfPot', 'GammaRay'], ['MedRes', 'DeepRes']]},
 {'name': 'Porosity-Lime',
  'curves': [['SelfPot', 'GammaRay'], ['DenPorLime', 'NeutPorLime']]},
 {'name': 'Porosity-Sand',
  'curves': [['SelfPot', 'GammaRay'], ['DenPorSand', 'NeutPorSand']]},
 {'name': 'Porosity-Dol',
  'curves': [['SelfPot', 'GammaRay'], ['NeutPorDol', 'DenPorDol']]},
 {'name': 'LMR', 'curves': [['SelfPot', 'GammaRay']]},
 {'name': 'MiniPlot-Lime',
  'curves': [['SelfPot', 'GammaRay'],
   ['MedRes', 'DeepRes'],
   ['DenPorLime', 'NeutPorLime']]},
 {'name': 'MiniPlot-Sand',
  'curves': [['SelfPot', 'GammaRay'],
   ['MedRes', 'DeepRes'],
   ['DenPorSand', 'NeutPorSand']]},
 {'name': 'MiniPlot-Dol',
  'curves': [['SelfPot', 'GammaRay'],
   ['MedRes', 'DeepRes'],
   ['NeutPorDol', 'DenPorDol']]}]

In [91]:
curve_displ_new

[{'name': 'Son-Den', 'curves': [['SelfPot', 'GammaRay'], ['Sonic']]},
 {'name': 'Son-Shear', 'curves': [['SelfPot', 'GammaRay'], ['Sonic']]},
 {'name': 'Resistivity',
  'curves': [['SelfPot', 'GammaRay'], ['MedRes', 'DeepRes']]},
 {'name': 'Porosity-Lime',
  'curves': [['SelfPot', 'GammaRay'], ['DenPorLime', 'NeutPorLime']]},
 {'name': 'Porosity-Sand',
  'curves': [['SelfPot', 'GammaRay'], ['DenPorSand', 'NeutPorSand']]},
 {'name': 'Porosity-Dol',
  'curves': [['SelfPot', 'GammaRay'], ['NeutPorDol', 'DenPorDol']]},
 {'name': 'LMR', 'curves': [['SelfPot', 'GammaRay']]},
 {'name': 'MiniPlot-Lime',
  'curves': [['SelfPot', 'GammaRay'],
   ['MedRes', 'DeepRes'],
   ['DenPorLime', 'NeutPorLime']]},
 {'name': 'MiniPlot-Sand',
  'curves': [['SelfPot', 'GammaRay'],
   ['MedRes', 'DeepRes'],
   ['DenPorSand', 'NeutPorSand']]},
 {'name': 'MiniPlot-Dol',
  'curves': [['SelfPot', 'GammaRay'],
   ['MedRes', 'DeepRes'],
   ['NeutPorDol', 'DenPorDol']]}]